### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players



In [2]:
#group by unique screennames
unique_players = purchase_data.groupby('SN').nunique()

#count screen names
NumberOfPlayers = int(unique_players['SN'].count())

#create list
Players_Array = {'Total Players': [NumberOfPlayers]}

#create dataframe
display_df_1 = pd.DataFrame(data=Players_Array)

#print to screen
display_df_1

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
#finds number of unique item IDs
number_of_items = purchase_data.groupby('Item ID').nunique()
NumberOfItems = int(number_of_items['Item ID'].count())

#finds unique item IDs so we can sum the prices
total_price = purchase_data.drop_duplicates('Item ID')
SumOfAllPrices = int(total_price['Price'].sum())

#divide sum of prices by number of items
AveragePrice = round((SumOfAllPrices / NumberOfItems), 2)

#format average price with dollar sign and comma seperator, comma unnessicary with the sample data provided but I'm alittle OCD
AveragePriceFormatted = "$" + str('{:,.2f}'.format(AveragePrice))

#find all unique purchase IDs
number_of_purchases = purchase_data.groupby('Purchase ID').nunique()
NumberOfPurchases = int(number_of_purchases['Purchase ID'].count())

#sum all prices together to get total revenue
SumOfAllPurchases = float(purchase_data['Price'].sum())

#format total revenue with dollar sign and comma seprator
SumOfAllPurchasesFormatted = "$" + str('{:,.2f}'.format(SumOfAllPurchases))

#put in a dict
Purchasing_Information = {
    "Number of Unique Items": [NumberOfItems],
    "Average Price": [AveragePriceFormatted],
    "Number of Purchases": [NumberOfPurchases],
    "Total Revenue": [SumOfAllPurchasesFormatted]
}

#put in dataframe
Finanical_Info = pd.DataFrame(Purchasing_Information)
Finanical_Info

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.04,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#group data by genders
players_gender = purchase_data.groupby('Gender').nunique()

#slice off everything except gendered count of unique screennames
players_gender_unique = players_gender.loc[:, ['SN']]

#find how many different genders there, this is mostly so I can iterate through and have the values match
number_of_genders = len(players_gender_unique.index)

#prepare empty list
percents = []

#for iterating
i = 0

#go through dataframe, for each row, figure the percentage, and format, then add to the list and iterate to next
while i < (number_of_genders):
    percents.append(str(round((float(players_gender_unique.iloc[i, 0])/NumberOfPlayers)*100,2)) + "%")
    i = i + 1
    
#add list to DF
players_gender_unique["Percentage of Players"]=percents

#print to screen
players_gender_unique

,SN,Percentage of Players
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#for formatting values into currency
def kev_currency_format(z):
    return "${0:.2f}".format(z)

#group by gender
players_gender_2 = purchase_data.groupby('Gender')

#get number of unique screennames, grouped by gender, into a dataframe
gender_purchase_count = players_gender["Purchase ID"]
gender_purchase_count_df = pd.DataFrame(data=gender_purchase_count)

#sum purchases by gender
gender_total_purchase = players_gender_2.sum()[["Price"]]

#merge our two new dataframes
average_price_df = gender_purchase_count_df.join(gender_total_purchase, how='outer')

#calculate average purchase price per purchase
average_price_df["Result"] = average_price_df["Price"] / average_price_df["Purchase ID"]

#pulls number of unique users by gender into the data table
average_price_df["More Info"] = players_gender_unique["SN"]

#figures average purchase per person
average_price_df["Avg Total Purchase per Person"] = average_price_df["Price"] / average_price_df["More Info"]

#formats everything like currency
average_price_df["Price"] = average_price_df["Price"].apply(kev_currency_format)
average_price_df["Result"] = average_price_df["Result"].apply(kev_currency_format)
average_price_df["Avg Total Purchase per Person"] = average_price_df["Avg Total Purchase per Person"].apply(kev_currency_format)

#a new clean dataframe for printing to screen
average_price_df_clean_for_display = pd.DataFrame()

#applying
average_price_df_clean_for_display["Purchase Count"] = average_price_df["Purchase ID"]
average_price_df_clean_for_display["Average Purchase Price"] = average_price_df["Result"]
average_price_df_clean_for_display["Total Purchase Value"] = average_price_df["Price"]
average_price_df_clean_for_display["Avg Total Purchase per Person"] = average_price_df["Avg Total Purchase per Person"]

#printing to screen
average_price_df_clean_for_display

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
#formats as with percentage sign for cleaner formating
def percentage_format(v):
    return f"{v}%"

#establishes the age bins
age_bins = [0, 9.5, 14.5, 19.5, 24.5, 29.5, 34.5, 39.5, 99]

#establishes bin labels
bin_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#going to slice down purchase_data to the two columns we need
age_columns = ["SN", "Age"]
unique_players_2 = purchase_data[age_columns]

#finds only unique SNs
unique_players_3=unique_players_2.drop_duplicates(subset="SN", keep="first")

#a new DataFrame for holding our data
age_df = pd.DataFrame()

#binning into the age bins
age_df["Ages"] = pd.cut(unique_players_3["Age"], age_bins, right=True, labels=bin_labels)
age_df_binned = age_df.Ages.value_counts(sort=False)

#cleaning up into a new dataframe for printing to screen
age_df_2 = pd.DataFrame()
age_df_2["Total Count"] = age_df_binned

#calculating the percentage and cleaning it up
age_df_2["Percentage of Players"] = round((age_df_2["Total Count"] / NumberOfPlayers) * 100, 2)
age_df_2["Percentage of Players"] = age_df_2["Percentage of Players"].apply(percentage_format)

#printing to screen
age_df_2

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
#create some blank DataFrames for moving our data around and displaying
purchase_data_age = pd.DataFrame()
purchase_data_age_binned = pd.DataFrame()
purchase_data_age_display = pd.DataFrame()
total_purchase_by_age = pd.DataFrame()

#create a copy of the data for binning
purchase_data_age = purchase_data

#organize into age bins
purchase_data_age["Ages"] = pd.cut(purchase_data["Age"], age_bins, right=True, labels=bin_labels)

#sum all purchases by age bin
total_purchase_by_age = purchase_data_age.groupby(["Ages"]).sum()
purchase_data_age_binnned = purchase_data_age.Ages.value_counts(sort=False)

#add to our display DF
purchase_data_age_display["Purchase Count"] = purchase_data_age_binnned

#this is just to hold the place so the second column is the average purchase price, there may be a more elegant way to move the columns around but this is a fast and dirty way
purchase_data_age_display["Average Purchase Price"] = purchase_data_age_display["Purchase Count"]

#more adding to the display DF
purchase_data_age_display["Total Purchase Value"] = total_purchase_by_age["Price"]

#figures average price per purchase
purchase_data_age_display["Average Purchase Price"] = purchase_data_age_display["Total Purchase Value"] / purchase_data_age_display["Purchase Count"]

#figures average price per person
purchase_data_age_display["Avg Total Purchase per Person"] = purchase_data_age_display["Total Purchase Value"] / age_df_2["Total Count"]

#format the dollar amounts as currency
purchase_data_age_display["Average Purchase Price"] = purchase_data_age_display["Average Purchase Price"].apply(kev_currency_format)
purchase_data_age_display["Total Purchase Value"] = purchase_data_age_display["Total Purchase Value"].apply(kev_currency_format)
purchase_data_age_display["Avg Total Purchase per Person"] = purchase_data_age_display["Avg Total Purchase per Person"].apply(kev_currency_format)

#print to display
purchase_data_age_display

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
#some blank DataFrames
spender_df_display = pd.DataFrame()
print_to_screen = pd.DataFrame()

#slice purchase_data down to what we need
spender_columns = ["Purchase ID", "SN", "Price"]
spenders_df = purchase_data[spender_columns]

#group SNs by purchases
spender_df_display["Purchase Count"] = spenders_df.groupby(by=["SN"])["Purchase ID"].count()

#placeholder
spender_df_display["Average Purchase Price"] = spender_df_display["Purchase Count"]

#sum total purchases by SN
spender_df_display["Total Purchase Price"] = spenders_df.groupby(by=["SN"])["Price"].sum()

#figure average purchases by SN
spender_df_display["Average Purchase Price"] = spender_df_display["Total Purchase Price"] / spender_df_display["Purchase Count"]

#find the five top spenders, orginially I tried to do this entirely in the spender_df_display DF but had issues with sorting the strings so just created a second DF
print_to_screen = spender_df_display.nlargest(5, "Total Purchase Price")

#format currency
print_to_screen["Total Purchase Price"] = print_to_screen["Total Purchase Price"].apply(kev_currency_format)
print_to_screen["Average Purchase Price"] = print_to_screen["Average Purchase Price"].apply(kev_currency_format)

#print to screen
print_to_screen

,Purchase Count,Average Purchase Price,Total Purchase Price
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [36]:
most_popular_items = pd.DataFrame()

most_popular_item_columns = ["Purchase ID", "Item ID", "Item Name", "Price"]

most_popular_items = purchase_data[most_popular_item_columns]

most_popular_items_display = pd.DataFrame()

most_popular_items_display = purchase_data.groupby(["Item ID", "Item Name"]).sum()

most_popular_items_display

#most_popular_items

#spender_df_display["Purchase Count"] = spenders_df.groupby(by=["SN"])["Purchase ID"].count()


,,Purchase ID,Age,Price
Item ID,Item Name,,,
0,Splinter,1149,74,5.12
1,Crucifer,1523,89,11.77
2,Verdict,2006,143,14.88
3,Phantomlight,2087,155,14.94
4,Bloodlord's Fetish,1885,124,8.50
...,...,...,...,...
178,"Oathbreaker, Last Hope of the Breaking Storm",4960,287,50.76
179,"Wolf, Promise of the Moonwalker",1948,126,26.88
181,Reaper's Toll,1278,109,8.30


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

